
  <h1>Import necessary libraries.</h1>

In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
import requests 
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import numpy as np
try:
    import folium
except:
    !pip install folium
    import folium
import os

# Download dataframe.
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

--2020-11-22 02:35:48--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.194, 169.63.96.176
Connecting to cocl.us (cocl.us)|169.63.96.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-11-22 02:35:49--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-11-22 02:35:49--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]


  <h1> Get data from web-page and parse it </h1>

In [2]:
siteData = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
dataSoup = BeautifulSoup(siteData,"html.parser")

  <h1> Columns </h1>

In [3]:
postalCodes = []
boroughs = []
neighborhoods = []

  <h1>Find Necessary elements in web page. </h1>

In [4]:
table = dataSoup.find('table')
for element in table.find_all("tr"):
    cells = element.find_all('td')
    if len(cells) > 0:
        postalCodes.append(cells[0].text[:-1])
        boroughs.append(cells[1].text[:-1])
        neighborhoods.append(cells[2].text[:-1])

  <h1> Create dataframe. </h1>

In [5]:
torontoDf = pd.DataFrame({"Postal Code":postalCodes,
                          "Borough":boroughs,
                          "Neighborhood":neighborhoods,
                         })

# Remove rows with  missing borough data.
torontoDf = torontoDf[torontoDf.Borough != 'Not assigned'].reset_index(drop=True)
torontoDf = torontoDf.groupby(["Postal Code","Neighborhood"], as_index=False).agg(lambda x: ", ".join(x))

# Assign borough name to neighborhood if neighborhood name is not assigned.
for index, row in torontoDf.iterrows():
    if row["Neighborhood"] == "Not Assigned":
        row["Neighborhood"] = row["Borough"]
    
torontoDf.head()

,Postal Code,Neighborhood,Borough
0,M1B,"Malvern, Rouge",Scarborough
1,M1C,"Rouge Hill, Port Union, Highland Creek",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


  <h1>Coordinates dataframe.</h1>

In [6]:
# Convert it to a pandas dataframe.
coordinatesDf = pd.read_csv("Geospatial_Coordinates.csv")
coordinatesDf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


  <h1> Merge dataframes.</h1>

In [7]:
torontoDf = pd.merge(torontoDf, coordinatesDf , on = "Postal Code")
torontoDf.head()

,Postal Code,Neighborhood,Borough,Latitude,Longitude
0,M1B,"Malvern, Rouge",Scarborough,43.806686,-79.194353
1,M1C,"Rouge Hill, Port Union, Highland Creek",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711
3,M1G,Woburn,Scarborough,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476


  <h1> Pin each neighborhood to map. </h1>

In [8]:
# Create the map.
torontoCoordinates = [43.6532,-79.3832]
torontoMap = folium.Map(location = torontoCoordinates, zoom_start = 12)

# Create a marker for each neighborhood on the dataframe.
for index,row in torontoDf.iterrows():
    folium.CircleMarker(
    location = [row["Latitude"],row["Longitude"]],
    radius = 5,
    popup = row["Neighborhood"],
    color = "blue",
    fill = True,
    fill_color = "dark-blue").add_to(torontoMap)
    
torontoMap

  <h1> Clustering each neighborhood to a cluster. </h1>

In [22]:
# The code was removed by Watson Studio for sharing.

In [10]:
# Getting nearby venues for each neighborhoood.

def getVenuesNearby(hoods,latitudes,longitudes,radius = 500):
    venuesList = []
    for name, lat, long in zip(hoods, latitudes, longitudes):
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        clientId,
        clientSecret,
        version,
        lat,
        long,
        500,
        limit)
    
        # Get JSON file for result.
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # Add each venue to list.
        venuesList.append([(
                name, 
                lat, 
                long, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
    nearbyVenues = pd.DataFrame([item for venueList in venuesList for item in venueList])
    nearbyVenues.columns = ['Neighborhood', 
                      'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
    return (nearbyVenues)
    

In [11]:
venueDf = getVenuesNearby(torontoDf["Neighborhood"],torontoDf["Latitude"],torontoDf["Longitude"])
venueDf.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


  <h1> Converting venue categories to binary values. </h1>

In [12]:
oneHotVenues = pd.get_dummies(venueDf["Venue Category"], prefix = "" ,prefix_sep = "")
oneHotVenues.drop("Neighborhood",1,inplace = True)
oneHotVenues.insert(0,"Neighborhood",venueDf["Neighborhood"])
oneHotVenues.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Road,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,

In [13]:
# Get dataset of the grouped venues by neighborhood and have their frequency be shown.
oneHotGrouped = oneHotVenues.groupby("Neighborhood").mean().reset_index()

In [14]:
def getCommonVenues(hood,n):
    
    # Sort the row.
    rowCategories = hood.iloc[1:]
    rowCategoriesSorted = rowCategories.sort_values(ascending = False)
    return rowCategoriesSorted.index.values[0:n]

  <h1> Showing the most popular venue categories for each neighborhood. </h1>

In [15]:
suffixes = ["st","nd","rd"]
columns = ["Neighborhood"]
topN = 10

# Create columns for top N venue types.
for n in range(topN):
    try:
        columns.append("{}{} most common venue type".format(n+1,suffixes[n]))
    except:
        columns.append("{}th most common venue type".format(n+1))

# Create new dataframe with most common venue types.
commonVenuesDf = pd.DataFrame(columns = columns)
commonVenuesDf["Neighborhood"] = oneHotGrouped["Neighborhood"]

# Fill the dataframe with the most frequent venue types from OneHotVenues.
for row in range(oneHotGrouped.shape[0]):
    commonVenuesDf.iloc[row,1:] = getCommonVenues(oneHotGrouped.iloc[row,:],topN)

# Show new datafreame.
commonVenuesDf.head()

,Neighborhood,1st most common venue type,2nd most common venue type,3rd most common venue type,4th most common venue type,5th most common venue type,6th most common venue type,7th most common venue type,8th most common venue type,9th most common venue type,10th most common venue type
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Clothing Store,Ethiopian Restaurant,Event Space,Escape Room,Electronics Store,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Pub,Pharmacy,Gym,Greek Restaurant,Discount Store,Department Store,Dessert Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Deli / Bodega,Shopping Mall,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Greek Restaurant,Thai Restaurant,Liquor Store,Comfort Food Restaurant,Juice Bar,Butcher,Café


  <h1> Clustering each neighborhood. </h1>

In [16]:
nClusters = 5

# Clearing non-numeric values for clustering
clusterVenueDf = oneHotGrouped.drop("Neighborhood",1)

# K-means model.
kMeans = KMeans(n_clusters = nClusters,random_state = 3).fit(clusterVenueDf)

In [18]:
# Append cluster label to dataframe
# commonVenuesDf.drop("Cluster",1,inplace = True)
commonVenuesDf.insert(0,'Cluster',kMeans.labels_)

# Merge the dataframes.
totalTorontoDf = torontoDf
totalTorontoDf = totalTorontoDf.join(commonVenuesDf.set_index("Neighborhood"), on = "Neighborhood")
droppedNa = totalTorontoDf.dropna(0,inplace = False)
totalTorontoDf.head()

,Postal Code,Neighborhood,Borough,Latitude,Longitude,Cluster,1st most common venue type,2nd most common venue type,3rd most common venue type,4th most common venue type,5th most common venue type,6th most common venue type,7th most common venue type,8th most common venue type,9th most common venue type,10th most common venue type
0,M1B,"Malvern, Rouge",Scarborough,43.806686,-79.194353,4.0,Fast Food Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,M1C,"Rouge Hill, Port Union, Highland Creek",Scarborough,43.784535,-79.160497,0.0,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711,2.0,Intersection,Breakfast Spot,Electronics Store,Restaurant,Medical Center,Rental Car Location,Bank,Mexican Restaurant,Dog Run,Discount Store
3,M1G,Woburn,Scarborough,43.770992,-79.216917,2.0,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Drugstore,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476,2.0,Caribbean Restaurant,Fried Chicken Joint,Bank,Gas Station,Athletics & Sports,Thai Restaurant,Bakery,Hakka Restaurant,Eastern European Restaurant,Dumpling Restaurant


  <h1> Plot the clusters to a map </h1>

In [23]:
clusteredMap = folium.Map(location = torontoCoordinates, zoom_start = 11)    
colorsList = ['maroon',"forestgreen","darkblue","slategrey","darkorange"]

# Plot each neighborhood from totalTorontoDf and it's color accordingly.
for hood, lat, long, clus in zip(droppedNa["Neighborhood"],droppedNa["Latitude"],droppedNa["Longitude"],droppedNa["Cluster"]):
    folium.CircleMarker(
    location = [lat,long],
    radius = 5,
    popup = "{}, Cluster {}".format(hood,int(clus)),
    color = colorsList[int(clus)],
    fill = True,
    fill_color = colorsList[int(clus)],
    fill_opacity = .7).add_to(clusteredMap)

# Show map.
clusteredMap
                         
                    

 <h1> Summary </h1>
 <h2> As you could see, The city of Toronto has an abundance of restaurants. Almost each neighborhood offers you a variety of cuisines.
      Thank you - 
    Imron Djalilov</h2>
    

In [20]:
totalTorontoDf.shape

(103, 16)